In [1]:
push!(LOAD_PATH, "../src")
using Quaternions
using FixedSizeArrays
using RigidBodyDynamics

In [2]:
mechanism = Mechanism{Float64}("world");
world = root(mechanism)

RigidBodyDynamics.RigidBody{Float64}(RigidBodyDynamics.CartesianFrame3D("world"),#undef)

In [3]:
body1 = RigidBody(rand(SpatialInertia{Float64}, CartesianFrame3D("body1")));
joint1 = Joint("1", QuaternionFloating());
joint1ToWorld = rand(Transform3D{Float64}, joint1.frameBefore, world.frame)
body1ToJoint1 = rand(Transform3D{Float64}, body1.frame, joint1.frameAfter)
attach!(mechanism, world, joint1, joint1ToWorld, body1, body1ToJoint1);

In [4]:
joint2 = Joint("2", Revolute(rand(Vec{3, Float64})));
body2 = RigidBody(rand(SpatialInertia{Float64}, joint2.frameAfter));
joint2ToBody1 = rand(Transform3D{Float64}, joint2.frameBefore, body1.frame)
attach!(mechanism, body1, joint2, joint2ToBody1, body2);

In [5]:
x = MechanismState{Float64}(mechanism);
zero_configuration!(x)

In [6]:
num_positions(x)

8

In [7]:
num_velocities(x)

7

In [8]:
cache = FrameCache(mechanism, x);

In [9]:
relative_transform(cache, joint2.frameAfter, body1.frame)

RigidBodyDynamics.Transform3D{Float64}(RigidBodyDynamics.CartesianFrame3D("after_2"),RigidBodyDynamics.CartesianFrame3D("body1"),0.19070818188597052 + 0.2928864760479799im + 0.08544561374269824jm + 0.9330310544682192km,Vec(0.4833238504560863,0.8121040155990059,0.22593786518570796))

In [10]:
center_of_mass(mechanism, cache)

RigidBodyDynamics.Point3D{Float64}(RigidBodyDynamics.CartesianFrame3D("world"),Vec(0.7661161419589212,0.005480038374100206,1.2930051430359706))

In [11]:
mass(mechanism)

0.15765187237826783

In [12]:
transform(cache, Point3D{Float64}(joint2.frameAfter), root(mechanism).frame)

RigidBodyDynamics.Point3D{Float64}(RigidBodyDynamics.CartesianFrame3D("world"),Vec(0.2213485686541812,0.24202525895893418,1.2432202461214843))

In [13]:
motion_subspace(joint1, x.q[joint1])

LoadError: LoadError: MethodError: `motion_subspace` has no method matching motion_subspace(::RigidBodyDynamics.Joint, ::Array{Float64,1}, ::RigidBodyDynamics.QuaternionFloating)
Closest candidates are:
  motion_subspace{T1,T2}(::RigidBodyDynamics.Joint, ::Array{T1,1}, ::Union{RigidBodyDynamics.Prismatic{T2},RigidBodyDynamics.QuaternionFloating,RigidBodyDynamics.Revolute{T2}})
  motion_subspace{T1}(::RigidBodyDynamics.Joint, ::Array{T1,1})
while loading In[13], in expression starting on line 1